# Cottrell Simulations
Based on the Fortran code found in appendix B of "Electrochemical Methods" by Bard and Faulkner

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Constants 
l = 100
DMA = 0.40
DMB = DMA

fa_old = 1
fa_new = 1
fa_old = 0
fa_new = 1